<a href="https://colab.research.google.com/github/isarandi/nlf/blob/main/demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# !pip install torch torchvision matplotlib opencv-python

  Using cached opencv_python-4.12.0.88-cp37-abi3-macosx_13_0_arm64.whl.metadata (19 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached mpmath-1.3.0-py3-none-any.whl.metadata (8.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 MB 11.3 MB/s  0:00:06m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 10.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 11.9 MB/s  0:00:00 eta 0:00:01
Using cached opencv_python-4.12.0.88-cp37-abi3-macosx_13_0_arm64.whl (37.9 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 13.4 MB/s  0:00:00 eta 0:00:01
Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 11.8 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 6.4 MB/s  0:00:00 eta 0:00:01m
Using cached sympy-1.14.0-py3-none-any.whl (6.3 MB)
Using cached mpmath-1.3.0-py3-no

In [1]:
# PyTorch version

import os
import torch
import torchvision  # Must import this for the model to load without error

!mkdir -p models
!wget -q -O models/nlf_l_multi.torchscript https://bit.ly/nlf_l_pt


zsh:1: command not found: wget


In [2]:
import os
print(os.getcwd())
print(os.path.exists('models/nlf_l_multi.torchscript'))

/Users/lemon/Documents/TUD/Thesis/Code/nlf
True


In [10]:
# device = ("cuda" if torch.cuda.is_available()
#           else "mps" if torch.backends.mps.is_available()
#           else "cpu")
device = "cpu"
print(f"Using device: {device}")

model = torch.jit.load("models/nlf_l_multi.torchscript", map_location=device).eval()

image = torchvision.io.read_image("data/dance.jpg").to(device)
frame_batch = image.unsqueeze(0) # [BatchSize, 3, H, W]
print(frame_batch.shape)

with torch.inference_mode():
   pred = model.detect_smpl_batched(frame_batch, model_name='smplx')

# SMPL Parametric predictions
pred['pose'], pred['betas'], pred['trans']
pred['joints3d'], pred['vertices3d']
pred['joints2d'], pred['vertices2d']

# Nonparametric joints and vertices
pred['joints3d_nonparam'], pred['vertices3d_nonparam']
pred['joints2d_nonparam'], pred['vertices2d_nonparam']
pred['joint_uncertainties'], pred['vertex_uncertainties']

Using device: cpu


([tensor([[ 60.7656,  89.9622,  92.9208,  74.5307, 386.2579, 318.8035,  78.8238,
           424.7066, 346.9449,  85.7797, 509.8427, 402.5501,  86.0616,  87.1459,
            85.7746, 121.8617, 121.1164, 114.4397, 179.6935, 240.0713, 210.9277,
           327.4243, 122.8263, 147.7758, 157.5511, 225.1087, 227.5220, 237.8588,
           228.7527, 228.8388, 238.9129, 234.6281, 238.8578, 244.3784, 231.6965,
           230.9342, 244.9237, 212.8060, 220.8098, 226.1958, 328.7358, 335.7325,
           345.7538, 330.6998, 344.6271, 352.2048, 304.8215, 318.8441, 333.1533,
           318.6633, 336.6500, 341.0136, 322.1026, 320.2289, 327.4242]])],
 [tensor([[119.1146, 119.5124, 121.2356,  ..., 158.3736, 156.0291, 156.6171]])])

In [7]:
# !pip install smplx

In [8]:
import smplx

In [16]:
import os
bm_root = os.getenv('DATA_ROOT', default='.')
bm = smplx.SMPLX(os.path.join(bm_root, 'smplx', 'SMPLX_NEUTRAL.npz'), use_pca=False).eval()

pose = pred['pose'][0]
betas = pred['betas'][0]
transl = pred['trans'][0]
res = bm(global_orient=pose[:, :3],
   body_pose=pose[:, 3:22*3],
   betas=betas,
   transl=transl,
   left_hand_pose=pose[:, 25*3:40*3],
   right_hand_pose=pose[:, 40*3:55*3],
   jaw_pose=pose[:, 22*3:23*3],
   leye_pose=pose[:, 23*3:24*3],
   reye_pose=pose[:, 24*3:25*3]
)

In [17]:
fov_degrees = 55
K = ptu3d.intrinsic_matrix_from_field_of_view(fov_degrees, image.shape[1:3])
def project_vertices(coords3d, intrinsic_matrix):
    projected = coords3d / torch.maximum(
         torch.tensor(0.001), torch.tensor(coords3d[..., 2:]))
    return torch.einsum('bnk,bjk->bnj', projected, intrinsic_matrix[..., :2, :])

NameError: name 'ptu3d' is not defined

In [83]:
%matplotlib notebook
import matplotlib.pyplot as plt
import numpy as np
import cameralib

fig, ax = plt.subplots()
cam = cameralib.Camera.from_fov(55, image.shape[1:3])
#verts2d = cam.world_to_image(pred['vertices3d'][0][0].detach().cpu().numpy())

#verts2d = pred['vertices2d'][0][0].detach().cpu().numpy()
verts2d = project_vertices(pred['vertices3d'][0].detach().cpu(), K).squeeze(0).numpy()

ax.imshow(image.cpu().numpy().transpose(1, 2, 0))
ax.scatter(*verts2d.T, s=1, c='r', alpha=0.5)
#ax.scatter(*v2d.detach().cpu().numpy().T, s=1, c='r', alpha=0.5)

/tmp/ipykernel_168364/4199168183.py:6: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots()


<IPython.core.display.Javascript object>

/tmp/ipykernel_168364/1601311042.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(0.001), torch.tensor(coords3d[..., 2:]))


In [52]:
image.shape

torch.Size([3, 1080, 1920])

In [85]:
! echo $PWD

/home/sarandi/rwth-home2/pose/pycharm/nlf


In [86]:
import torch
import torchvision  # Must import this for the model to load without error

device = "cuda:1"

model = torch.jit.load('models/nlf_l_multi_0.2.2.torchscript').to(device).eval()
frame_batch = torch.from_numpy(image).permute(2, 0, 1).unsqueeze(0).to(device)

with torch.inference_mode(), torch.device('cuda:1'):
    pred = model.detect_smpl_batched(frame_batch)


RuntimeError: CUDA error: invalid device ordinal
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.


In [87]:
import torch

class Module(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = torch.nn.Linear(10, 10)
        self.buf = torch.nn.Buffer(torch.randn(10, 10))

    def forward(self, x):
        return self.linear(x)

In [88]:
m = Module()

In [92]:
list(m.buffers())

[tensor([[ 0.9588],
         [ 0.1744],
         [-0.1770],
         [-0.8480],
         [-0.0126],
         [ 0.5749],
         [-0.3269],
         [ 0.1904],
         [-0.2358],
         [ 0.3534]])]

In [91]:
m.buf = torch.nn.Buffer(torch.randn(10, 1))

In [108]:
list(a.named_buffers())

[]

In [122]:

a = torch.jit.load('/home/sarandi/rwth-home2/pose/git_checkouts/ultralytics/yolov8x.torchscript').cpu()

In [125]:

a(torch.randn(1, 3, 640, 640).cpu())

RuntimeError: The following operation failed in the TorchScript interpreter.
Traceback of TorchScript, serialized code (most recent call last):
  File "code/__torch__/ultralytics/nn/tasks.py", line 74, in forward
    _35 = (_18).forward(act, _34, )
    _36 = (_20).forward((_19).forward(act, _35, ), _29, )
    _37 = (_22).forward(_33, _35, (_21).forward(act, _36, ), )
           ~~~~~~~~~~~~ <--- HERE
    return _37
  File "code/__torch__/ultralytics/nn/modules/head.py", line 45, in forward
    anchor_points = torch.to(torch.unsqueeze(CONSTANTS.c0, 0), dtype=6, layout=0, device=torch.device("cuda:0"))
    lt, rb, = torch.chunk(_13, 2, 1)
    x1y1 = torch.sub(anchor_points, lt)
           ~~~~~~~~~ <--- HERE
    x2y2 = torch.add(anchor_points, rb)
    c_xy = torch.div(torch.add(x1y1, x2y2), CONSTANTS.c1)

Traceback of TorchScript, original code (most recent call last):
/home/sarandi/rwth-home2/pose/git_checkouts/ultralytics/ultralytics/utils/tal.py(312): dist2bbox
/home/sarandi/rwth-home